# Generating a Language with Machine Learning

The following notebook will experiment with generating a language using neural networks and generative deep learning. This is, by no means, a production-ready system, nor is it a complete network; rather, the purpose of this experiment is to see what is possible with creating a language.

In [ ]:
import numpy as np
from tensorflow import keras